# we need a fast NN algo for searching embedding space

so we test!

In [1]:
import torch
from pynndescent import NNDescent
import time

W_E = torch.randn((1000, 896), requires_grad=False)
index = NNDescent(W_E)
index.prepare()




OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [3]:
q = torch.randn((3, 896), requires_grad=False)
# make a qury
ind, dist = index.query(q, k=3)
display(ind)
display(dist)

array([[100, 414, 772],
       [200, 341, 769],
       [341,  77, 851]], dtype=int32)

array([[40.257603, 40.323463, 40.52182 ],
       [38.149002, 38.591843, 39.063892],
       [39.64547 , 39.984737, 40.41433 ]], dtype=float32)

In [ ]:
device = "mps"

def nearest_neighbour_loss(x, index, W_E):

    # x should be of shape (N, 896)
    ind, dist = index.query(x.detach(), k=3)
    # get the first column of index
    ind = ind[:, 0].flatten()
    # we then form another matrix, with the nearest neighbors
    # concretely, we concatenate W_E[i] for i in ind
    nearest_neighbour = W_E[ind].detach().to(device)
    # we then calculate the distance between x and nearest_neighbour for each row in both
    distance = torch.norm(x - nearest_neighbour, p=2, dim=1)
    return torch.sum(distance)